# 代码测试工具

Javascript语言本身可以说是个残疾人,连基本的代码测试工具都没有,我们无法像在用python时那样简单的使用assert unittest这些语言自带工具或者标准包来做代码测试,测试工具几乎都是第三方的.

## 错误

### 基本错误

js也有一套简单的错误系统和错误处理机制,但很不好用,基本的错误包括:


+ Error 错误的基类
+ AssertionError 断言错误
+ RangeError 超出容器范围错误
+ ReferenceError 变量未定义错误
+ SyntaxError 语法错误
+ TypeError 类型错误
+ EvalError `eval()`函数发生的错误
+ URIError 浏览器中访问地址出错

通常用js的不会自己定义错误,但需要定义的话也和python一样,继承就好了

### 抛出错误

js使用语法

```js
throw new Error(msg)
```

来抛出错误

### 错误捕捉

js的错误捕捉语法是:

```js
try{
    block1
}catch (error){
    block2
}finally{
    block2
}
```

这个语法和python的基本一一对应


## 断言
js并没有断言语句,但node环境有一个简单的标准库`assert`,可以使用

In [1]:
import assert from "assert"

In [4]:
assert.equal(1,0)

AssertionError [ERR_ASSERTION]: 1 == 0

这个工具相当简单,但多数时候也够用了.

我们可以使用[chai](http://chaijs.com/)来实现断言功能

安装依然是npm安装

    npm install chai --save-dev

chai有三种断言风格

+ Assert风格

    类似python中的assert语句
    
    ```javascript
    var assert = chai.assert

    assert.typeOf(foo, 'string')
    assert.equal(foo, 'bar')
    assert.lengthOf(foo, 3)
    assert.property(tea, 'flavors')
    assert.lengthOf(tea.flavors, 3)
    ```
    这种风格代码最保守,我比较习惯用,标准库就有,就可以少装一个依赖
    
+ Expect

    推断方法,相对更加接近口语化,英语看来就是"我推测xxx是ooo"这样
    
    ```javascript
    var expect = chai.expect

    expect(foo).to.be.a('string')
    expect(foo).to.equal('bar')
    expect(foo).to.have.length(3)
    expect(tea).to.have.property('flavors').with.length(3)
    ```
    这种风格比较中庸,好像是最受欢迎的一种
    
+ Should
    
    最口语化得一种风格,意识是"xxx应该是ooo"
    
    ```javascript
    chai.should()

    foo.should.be.a('string')
    foo.should.equal('bar')
    foo.should.have.length(3)
    tea.should.have.property('flavors').with.length(3)
    ```
    这种风格不建议用,会修改原型

## 测试框架

做代码测试可以使用[mocha](https://mochajs.org/)来做


安装依然是npm

```shell
npm install mocha --save-dev
```

之后修改package.json的scripts字段即可

```json
{
    ...,
    "scripts":{
    ...
        "test": "./node_modules/.bin/mocha --require @babel/register --reporter Min"
     ...
     },
...
}

```

mocha的配置也可以写在`package.json`的`mocha`字段中,上面的参数可以改为如下设置:

```json
{
    ...,
    "scripts":{
        ...
        "test": "./node_modules/.bin/mocha"
        ...
     },
    ...,
    "mocha":{
         "require": "@babel/register",
         "reporter": "Min"
    },
    ...
}

```

mocha本身不提供不同方言包括高版本js语法的支持,要支持的话必须设置其`--require`参数,

+ babel需要设置`--require @babel/register`

+ typescript需要设置`--require ts-node/register`

这边我们定义的是最简报告输出,mocha支持的输出方式还包括其他选择,我们可以在[对应文档中查看到](https://mochajs.org/#reporters)

### 使用:


写测试代码:

```javascript
import assert from "assert"
describe('Array', function() {
  describe('#indexOf()', function () {
    it('should return -1 when the value is not present', function () {
      assert.equal(-1, [1,2,3].indexOf(5))
      assert.equal(-1, [1,2,3].indexOf(0))
    })
  })
})
```

上例中

+ describe(des,func)方法是一个测试的描述,一般一个describe()描述一个方法或类
+ it(des,func)方法则是一个测试具体的一个分支,一般一个it()描述一个方法的一种可能输入条件
+ mocha支持定义钩子,可以使用的钩子有:

    + before(func)定义某操作之前的操作
    + after(func) 定义某操作之后的操作
    + beforeEach(func)定义块内每个操作之前的操作 
    + afterEach(func) 定义块内每个操作之后的操作
    
+ mocha支持测试自定义操作
    
    + skip(des,func)跳过某测试
    + only(des,func)只进行某操作
    
要执行测试只要使用命令`npm run test`即可.


> 使用babel的例子

这个例子在分支[es-test](https://github.com/hszofficial/js-toolchain-exp/tree/es-test).我们定义了一个简单的模块来实验mocha的使用,

+ 被测代码`src/index.js`

    ```javascript
    const add = (a, b) => a + b

    class Child {
        constructor (name, age) {
            this.name = name
            this.age = age
        }
        self_introduction () {
            return `hello my name is ${ this.name }, I'm ${ this.age } years old`
        }
    }

    export { add, Child }
    ```
    
+ 测试代码`test/test.js`

    ```javascript
    import assert from 'assert'
    import { add, Child } from '../src/index.js'
    describe('add', function () {
        it('should return -1 when the value is 0,-1', function () {
            assert.equal(add(0, -1), -1)
        })
    })

    describe('child', function () {
        it('should return -1 when the value is 0,-1', function () {
            let tom = new Child("tom",8)
            assert.equal(tom.self_introduction(), "hello my name is tom, I'm 8 years old")
        })
    })
    ```
    
当我们执行`npm run test`的时候就可以看到输出了.

> 使用TypeScript的例子

这个例子在分支[ts-test](https://github.com/hszofficial/js-toolchain-exp/tree/ts-test).我们定义了一个简单的模块来实验mocha的使用.

要在typescript中使用mocha也需要安装它的类声明包`@types/mocha`

+ 被测代码`src/index.ts`

    ```typescript
    const add = (a: number, b: number): number => a + b

    class Child {
        public name: string
        public age: number
        constructor(name: string, age: number) {
            this.name = name
            this.age = age
        }
        self_introduction(): string {
            return `hello my name is ${this.name}, I'm ${this.age} years old`
        }
    }

    export { add, Child }
    ```
    
+ 测试代码`test/test.ts`

    ```typescript
    import * as assert from 'assert'
    import { add, Child } from '../src/index'
    describe('add', function () {
        it('should return -1 when the value is 0,-1', function () {
            assert.equal(add(0, -1), -1)
        })
    })

    describe('child', function () {
        it('should return -1 when the value is 0,-1', function () {
            let tom = new Child("tom", 8)
            assert.equal(tom.self_introduction(), "hello my name is tom, I'm 8 years old")
        })
    })
    ```
    
当我们执行`npm run test`的时候就可以看到输出了.

## 代码覆盖率

[istanbul(nyc)](https://istanbul.js.org/docs/tutorials/)是最常用的代码覆盖率工具. 

```bash
npm install --save-dev nyc
```

istanbul的配置也可以放在package.json中,使用字段`nyc`.


我们只要在`mocha`命令前面加上`nyc`即可实现代码覆盖率的统计,即:



> 使用babel的例子

`istanbul`并不支持直接为es6以上的代码提供覆盖率统计,需要额外安装babel的一个插件`babel-plugin-istanbul`来实现.
这个例子在分支[es-test_coverage](https://github.com/hszofficial/js-toolchain-exp/tree/es-test_coverage)下.
它是从上面的`es-test`分支中分出来的,我们只要安装好插件,然后修改下`package.json`中的配置即可:

```json
{
    ...,
    "nyc": {
        "require": [
          "@babel/register"
        ],
        "reporter": [
          "html"
        ],
        "sourceMap": false,
        "instrument": false
      },
    ...,
    "babel":{
        ...,
        "plugins": [
            "istanbul"
        ]
    },
    ...
}
```

然后我们修改`test`命令

```json
{
    ...,
    "scripts": {
    ...,
    "test": "./node_modules/.bin/nyc ./node_modules/.bin/mocha --require @babel/register"
    },
    ...
}
```

这样就可以输出覆盖率文档了,如果我们不需要html的文档,也可以使用`--reporter=text`

> 使用typescript的例子

我们的ts设置中通常会将`sourcemap`设置为true以加快调试效率,如果有这个设置的话,我们还需要额外安装插件`source-map-support`作为mocha的require参数.

```json
{
    ...,
    "scripts": {
        ...,
        "test": "./node_modules/.bin/nyc ./node_modules/.bin/mocha"
    },
    ...,
    "mocha": {
        "extension": [
            "ts"
        ],
        "require": [
            "ts-node/register",
            "source-map-support/register"
        ],
        "recursive": true
    },
    "nyc": {
        "extension": [
            ".ts",
            ".tsx"
        ],
        "exclude": [
            "**/*.d.ts"
        ],
        "reporter": [
            "html"
        ],
        "all": true
    },
    ...
}
```